## [Smallest Sufficient Team](https://leetcode.com/problems/smallest-sufficient-team/)
- Given:
    + `req_skills`: list of required skills
    + `people`: list of people, each person has a set of skills
- Find
    - `smallest sufficient team`
        + a set of people such that for every required skill in req_skills 
        + smallest possible size, represented by the index of each person.

- Constraints
    + 1 <= req_skills.length <= 16
    + 1 <= people.length <= 60
- Examples

```
Input: 
    req_skills = ["java","nodejs","reactjs"],
    people = [["java"],["nodejs"],["nodejs","reactjs"]]
Output: [0,2]
```

```
Input: 
    req_skills = ["algorithms","math","java","reactjs","csharp","aws"],
    people = [["algorithms","math","java"],["algorithms","math","reactjs"],["java","csharp","aws"],["reactjs","csharp"],["csharp","math"],["aws","java"]]
Output: [1,2]
```

#### Solution DP - bitmask + traceback

```Cpp
class Solution {
private:
    typedef unsigned long long ull;
    typedef pair<int,int> ii;
public:
    vector<int> smallestSufficientTeam(
            vector<string>& req_skills,
            vector<vector<string>>& people) {
        int N = req_skills.size();
        int P = people.size();

        // skills to int
        unordered_map<string, int> skill_to_int;
        for(int i=0; i<N; ++i) {
            skill_to_int[req_skills[i]] = i;
        }

        // dp[p][subset] = optimal (smallest) size
        // long dp[P+1][1<<N]
        vector<vector<long>> dp(P+1, vector<long>(1<<N, INT_MAX));
        dp[0][0] = 0;
        // traceback
        // pair<int,int> traceback[P+1][1<<N]
        vector<vector<ii>> trace(P+1, vector<ii>(1<<N, ii({0,0})));
        trace[0][0] = {0,0};

        for(int p=0; p<P; ++p) {
            // get person[p] skill set
            ull personal_skills = 0;
            for(string &skill:people[p]) {
                if(skill_to_int.count(skill) != 0) personal_skills |= (1 << skill_to_int[skill]);
            }

            // bitmask
            for(ull subset=0; subset<(1<<N); ++subset) {
                // Add p case
                if(dp[p+1][subset | personal_skills] > dp[p][subset] + 1) {
                    dp[p+1][subset | personal_skills] = dp[p][subset] + 1;
                    trace[p+1][subset | personal_skills] = {p, subset};
                }

                // Do not add p
                if(dp[p+1][subset] > dp[p][subset]) {
                    dp[p+1][subset] = dp[p][subset];
                    trace[p+1][subset] = {p, subset};
                }
            }
        }

        // traceback
        vector<int> ans;

        ii cur = {P, (1<<N)-1};
        ii pre = trace[cur.first][cur.second];
        while(cur != ii()) {
            if(cur.second != pre.second) ans.push_back(pre.first);

            cur = pre;
            pre = trace[cur.first][cur.second];
        }
        return ans;
    }
};
```

#### Solution - knapsack bitmask - Optimize space

```Cpp
class Solution {
private:
    typedef unsigned long long ull;
    typedef pair<int,int> ii;
public:
    vector<int> smallestSufficientTeam(
            vector<string>& req_skills,
            vector<vector<string>>& people) {
        int N = req_skills.size();
        int P = people.size();

        // skills to int
        unordered_map<string, int> skill_to_int;
        for(int i=0; i<N; ++i) {
            skill_to_int[req_skills[i]] = i;
        }

        // dp[p][subset] = optimal (smallest) size
        // int dp[2][1<<N]
        vector<vector<long>> dp(2, vector<long>(1<<N, INT_MAX));
        dp[0][0] = 0;
        // cache[p][subset] = optimal set of people (vector<int> with smallest size)
        // vector<int> cache[2][1<<N]
        vector<vector<vector<int>>> cache(2, vector<vector<int>>(1<<N, vector<int>()));

        for(int p=0; p<P; ++p) {
            // get person[p] skill set
            ull personal_skills = 0;
            for(string &skill:people[p]) {
                if(skill_to_int.count(skill) != 0) personal_skills |= (1 << skill_to_int[skill]);
            }

            // bitmask
            for(ull subset=0; subset<(1<<N); ++subset) {
                // Add p case
                if(dp[(p+1)%2][subset | personal_skills] > dp[p%2][subset] + 1) {
                    dp[(p+1)%2][subset | personal_skills] = dp[p%2][subset] + 1;

                    cache[(p+1)%2][subset | personal_skills] = cache[p%2][subset];
                    cache[(p+1)%2][subset | personal_skills].push_back(p);
                }

                // Do not add p
                if(dp[(p+1)%2][subset] > dp[p%2][subset]) {
                    dp[(p+1)%2][subset] = dp[p%2][subset];
                    cache[(p+1)%2][subset] = cache[p%2][subset];
                }
            }
        }
        return cache[P%2][(1<<N)-1];
    }
};
```

#### Solution DP map

```C++
class Solution {
private:
    typedef unsigned long long ull;
public:
    vector<int> smallestSufficientTeam(
            vector<string>& req_skills,
            vector<vector<string>>& people) {
        int N = req_skills.size();
        int P = people.size();

        // skills to int
        unordered_map<string, int> skill_to_int;
        for(int i=0; i<N; ++i) {
            skill_to_int[req_skills[i]] = i;
        }

        // dp[subset] = optimal set of people (vector<int> with smallest size)
        unordered_map<int, vector<int>> dp;
        dp.reserve(1 << N);
        dp[0] = {};

        for(int p=0; p<P; ++p) {
            // get person[p] skill set
            ull personal_skills = 0;
            for(string &skill:people[p]) {
                if(skill_to_int.count(skill) != 0) personal_skills |= (1 << skill_to_int[skill]);
            }

            // Update DP
            for(const auto &[subset, persons]:dp) {
                // Relax, add person p
                if(
                    dp.count(subset | personal_skills) == NULL ||
                    dp[subset | personal_skills].size() > dp[subset].size()+1
                ) {
                    dp[subset | personal_skills] = dp[subset];
                    dp[subset | personal_skills].push_back(p);
                }
            }
        }

        return dp[(1<<N)-1];
    }
};
```